In [3]:
import pickle
import jax
import jax.numpy as jnp
from dataclasses import asdict
import jax.tree_util as tree
import numpy as np
import torch.onnx

In [19]:
with open("/home/rajath/Documents/Go2_mjx/utils/weights/go2_params-5.pkl", "rb") as f:
    params_loaded = pickle.load(f)

# Convert back to JAX arrays if needed
params_jax = jax.tree.map(jnp.array, params_loaded)

print("Params successfully loaded")

Params successfully loaded


In [26]:
params_jax

(RunningStatisticsState(mean={'privileged_state': Array([-2.0893412e-02, -4.9301184e-04,  1.0069761e-02, -2.1216734e-03,
        -1.4544168e-03, -3.6662046e-04,  6.5686265e-03,  3.0592836e-03,
        -9.9798530e-01, -7.4864395e-02,  3.4941953e-02, -5.2248247e-02,
         5.6147832e-02,  5.5461660e-02, -1.9936912e-02, -6.4740419e-02,
         4.7848884e-02, -3.3162825e-02,  5.3562108e-02,  2.3655953e-02,
        -3.9569840e-02, -1.1007247e-02,  2.1750525e-02, -6.6112496e-02,
         5.8366489e-03,  2.0859608e-02, -5.6877889e-02, -7.0117926e-03,
         8.9179222e-03, -4.3150872e-02,  4.4544660e-03,  1.3599153e-02,
        -5.3807698e-02, -3.3322352e-01,  9.9399485e-02,  3.1194550e-01,
         2.9165128e-01,  8.2773201e-02,  2.1366736e-01, -3.6443084e-01,
         1.1011693e-01,  2.0681904e-01,  3.1304368e-01,  1.7040579e-01,
         2.6569417e-01,  4.5625670e-04, -1.3381366e-04,  3.0227285e-04,
        -2.1146704e-03, -1.4556652e-03, -3.5802033e-04, -8.1252448e-02,
        -3.0087

In [21]:
mean_std_all = asdict(params_loaded[0])

In [22]:
mean_std_all.keys()

dict_keys(['mean', 'std', 'count', 'summed_variance'])

In [30]:
mean_std_all['mean']['state'].shape

(48,)

In [23]:
weights_bias = params_loaded[1]

In [24]:
weights_bias.keys()

dict_keys(['params'])

In [25]:
weights_bias["params"].keys()

dict_keys(['hidden_0', 'hidden_1', 'hidden_2', 'hidden_3', 'hidden_4', 'hidden_5'])

In [9]:
weights_bias["params"]['hidden_0'].keys()

dict_keys(['bias', 'kernel'])

In [10]:
weights_bias["params"]['hidden_0']['bias'].shape

(512,)

In [31]:
weights_bias["params"]['hidden_3']['kernel'].shape

(256, 256)

In [14]:
weights_bias["params"]['hidden_0']['kernel']

array([[-1.10215895e-01,  2.74286196e-02, -8.43288600e-02, ...,
        -1.10462168e-02, -2.38835234e-02, -1.90371629e-02],
       [-1.05272226e-01,  1.52543813e-01, -2.29068864e-02, ...,
        -4.04473254e-03,  1.36429993e-02, -3.55478585e-01],
       [-2.50817567e-01,  3.62823345e-02,  4.08163041e-01, ...,
         4.65863049e-01,  4.50724095e-01,  1.22397244e-01],
       ...,
       [-4.23305899e-01, -5.37335455e-01,  3.28460634e-01, ...,
        -1.09513998e-01, -1.45895258e-01,  9.92573053e-03],
       [ 8.55722725e-01, -8.16386640e-02, -1.74530923e-01, ...,
         1.66195258e-01, -7.33975172e-01, -1.28235504e-01],
       [ 3.57994169e-01, -8.62310827e-02, -2.60293573e-01, ...,
        -3.33543688e-01, -2.93473113e-06, -8.60006064e-02]], dtype=float32)

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class MLP(nn.Module):
    def __init__(
        self,
        layer_sizes,
        activation=nn.ReLU(),
        kernel_init="lecun_uniform",
        activate_final=False,
        bias=True,
        layer_norm=False,
        mean_std=None,
    ):
        super().__init__()

        self.layer_sizes = layer_sizes
        self.activation = activation
        self.kernel_init = kernel_init
        self.activate_final = activate_final
        self.bias = bias
        self.layer_norm = layer_norm

        # Register mean and std as buffers (non-trainable parameters)
        if mean_std is not None:
            self.register_buffer("mean", torch.tensor(mean_std[0], dtype=torch.float32))
            self.register_buffer("std", torch.tensor(mean_std[1], dtype=torch.float32))
        else:
            self.mean = None
            self.std = None

        # Build the MLP block
        self.mlp_block = nn.Sequential()
        for i in range(len(self.layer_sizes) - 1):
            in_features = self.layer_sizes[i]
            out_features = self.layer_sizes[i + 1]

            # Add linear layer
            dense_layer = nn.Linear(in_features, out_features, bias=self.bias)
            self.mlp_block.add_module(f"hidden_{i}", dense_layer)

            # Initialize weights (e.g., Lecun uniform initialization)
            if self.kernel_init == "lecun_uniform":
                nn.init.kaiming_uniform_(dense_layer.weight, mode='fan_in', nonlinearity='relu')

            # Add layer normalization if enabled
            if self.layer_norm and i < len(self.layer_sizes) - 2:  # No layer norm after the last layer
                self.mlp_block.add_module(f"layer_norm_{i}", nn.LayerNorm(out_features))

            # Add activation function, except for the final layer if `activate_final` is False
            if i < len(self.layer_sizes) - 2 or self.activate_final:  # Add activation for all but the last layer
                self.mlp_block.add_module(f"activation_{i}", self.activation)

    def forward(self, inputs):
        # Handle list inputs
        if isinstance(inputs, list):
            inputs = inputs[0]

        # Normalize inputs if mean and std are provided
        if self.mean is not None and self.std is not None:
            inputs = (inputs - self.mean) / self.std

        # Pass through the MLP block
        logits = self.mlp_block(inputs)

        # Split the output into two parts and apply tanh to the first half
        loc, _ = torch.split(logits, logits.size(-1) // 2, dim=-1)
        return torch.tanh(loc)

def make_policy_network(
    observation_size,
    action_size,
    mean_std,
    hidden_layer_sizes=[256, 256],
    activation=nn.SiLU(),
    kernel_init="lecun_uniform",
    layer_norm=False,
):
    layers = hidden_layer_sizes + [action_size]
    print(layers)
    policy_network = MLP(
        layer_sizes= layers,
        activation=activation,
        kernel_init=kernel_init,
        layer_norm=layer_norm,
        mean_std=mean_std,
    )
    return policy_network

In [19]:
mean_std = (torch.tensor(mean_std_all['mean']['state']), torch.tensor(mean_std_all['std']['state']))

In [25]:
th_policy_network = make_policy_network(
    observation_size = 48,
    action_size = 12*2,
    mean_std=mean_std,
    hidden_layer_sizes=[48, 512, 256, 128])

[48, 512, 256, 128, 24]


/tmp/ipykernel_21801/1165096672.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("mean", torch.tensor(mean_std[0], dtype=torch.float32))
/tmp/ipykernel_21801/1165096672.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("std", torch.tensor(mean_std[1], dtype=torch.float32))


In [26]:
th_policy_network

MLP(
  (activation): SiLU()
  (mlp_block): Sequential(
    (hidden_0): Linear(in_features=48, out_features=512, bias=True)
    (activation_0): SiLU()
    (hidden_1): Linear(in_features=512, out_features=256, bias=True)
    (activation_1): SiLU()
    (hidden_2): Linear(in_features=256, out_features=128, bias=True)
    (activation_2): SiLU()
    (hidden_3): Linear(in_features=128, out_features=24, bias=True)
  )
)

In [27]:
# Access and modify weights and biases
for i, layer in enumerate(th_policy_network.mlp_block):
    if isinstance(layer, nn.Linear):
        print(f"Layer {i}:")
        print("Original weights:", layer.weight.shape)
        print("Original biases:", layer.bias.shape)

Layer 0:
Original weights: torch.Size([512, 48])
Original biases: torch.Size([512])
Layer 2:
Original weights: torch.Size([256, 512])
Original biases: torch.Size([256])
Layer 4:
Original weights: torch.Size([128, 256])
Original biases: torch.Size([128])
Layer 6:
Original weights: torch.Size([24, 128])
Original biases: torch.Size([24])


In [16]:
type(weights_bias["params"]['hidden_0'])

dict

In [17]:
for key, values in weights_bias["params"].items():
    print(f"layer:{key}------------------")
    for key, kernel_bias in values.items():
        print(f"{key} size: {kernel_bias.shape}")

layer:hidden_0------------------
bias size: (512,)
kernel size: (48, 512)
layer:hidden_1------------------
bias size: (256,)
kernel size: (512, 256)
layer:hidden_2------------------
bias size: (128,)
kernel size: (256, 128)
layer:hidden_3------------------
bias size: (24,)
kernel size: (128, 24)


In [18]:
for i, layer in enumerate(th_policy_network.mlp_block):
    print(i, layer)

0 Linear(in_features=48, out_features=512, bias=True)
1 SiLU()
2 Linear(in_features=512, out_features=256, bias=True)
3 SiLU()
4 Linear(in_features=256, out_features=128, bias=True)
5 SiLU()
6 Linear(in_features=128, out_features=24, bias=True)


In [28]:
# Assuming th_policy_network is already defined
values = [(key,value) for key, value in weights_bias["params"].items()]
j = 0
for i, layer in enumerate(th_policy_network.mlp_block):
    if isinstance(layer, nn.Linear):  # Check if the layer is a Linear layer
        #print(f"Layer {values[i][0]}:")
        #print("Weights shape:", layer.weight.shape)
        #print("Biases shape:", layer.bias.shape)
        #print(f"before: {layer.weight.data}, size: {layer.weight.data.shape}")
        #print(f"BEFOREvalues: {values[i][1]['kernel']}, size: {values[i][1]['kernel'].shape}")

        #print(f"size: {values[i][1]['bias'].shape}")
        #print(f"size: {values[i][1]['kernel'].shape}")

        if (i%2==0):
            
            print(layer)
            transpose_tensor_kernel = torch.tensor(values[j][1]['kernel']).t()
            transpose_tensor_bias = torch.tensor(values[j][1]['bias']).t()

            #print(f"ker: {transpose_tensor_kernel.shape}")
            #print(f"bia: {transpose_tensor_bias.shape}")
                # Assign new weights and biases
            layer.weight.data = transpose_tensor_kernel
            layer.bias.data = transpose_tensor_bias
            j = j + 1
     

Linear(in_features=48, out_features=512, bias=True)
Linear(in_features=512, out_features=256, bias=True)
Linear(in_features=256, out_features=128, bias=True)
Linear(in_features=128, out_features=24, bias=True)


In [29]:
batch_size = 1
input = torch.randn(batch_size, 48)  # For a batch of inputs

In [30]:
th_policy_network

MLP(
  (activation): SiLU()
  (mlp_block): Sequential(
    (hidden_0): Linear(in_features=48, out_features=512, bias=True)
    (activation_0): SiLU()
    (hidden_1): Linear(in_features=512, out_features=256, bias=True)
    (activation_1): SiLU()
    (hidden_2): Linear(in_features=256, out_features=128, bias=True)
    (activation_2): SiLU()
    (hidden_3): Linear(in_features=128, out_features=24, bias=True)
  )
)

In [31]:
input.shape

torch.Size([1, 48])

In [32]:
th_policy_network.forward(input)

tensor([[-0.6950, -0.0239, -0.3071, -0.1466, -0.7421,  0.9410, -0.6703,  0.2743,
          0.3922,  0.2304, -0.5998, -0.5471]], grad_fn=<TanhBackward0>)

In [23]:
mean_std

(tensor([-5.3753e-03, -3.7227e-03,  8.1214e-03,  2.5784e-03, -6.0854e-03,
         -1.6575e-03,  6.0498e-03, -1.8035e-03, -9.9622e-01,  9.2987e-05,
          2.4611e-02,  1.0910e-01,  1.6130e-02,  1.2811e-02,  9.1156e-02,
         -1.9285e-02,  9.3518e-03,  1.0188e-01,  3.8124e-02,  2.0091e-02,
          8.4244e-02, -4.6906e-03,  8.9083e-03, -2.5855e-02,  5.9413e-03,
          9.4363e-03, -2.6273e-02, -6.1056e-03,  4.4214e-03, -2.1119e-02,
          6.9455e-03, -1.0700e-03, -6.9315e-03, -2.3088e-01, -3.0316e-02,
          5.0119e-01,  2.5405e-01, -1.0623e-02,  5.2193e-01, -2.1492e-01,
          5.4053e-02,  4.6472e-01,  2.5415e-01,  3.8651e-02,  4.8790e-01,
         -2.9012e-03, -7.5254e-04, -1.1815e-04]),
 tensor([0.3682, 0.1376, 0.1472, 0.8580, 0.4785, 0.5599, 0.0573, 0.0689, 0.0445,
         0.1033, 0.1173, 0.1269, 0.1088, 0.1202, 0.1283, 0.0904, 0.1269, 0.1364,
         0.0931, 0.1213, 0.1330, 1.9139, 2.7987, 3.6095, 1.9457, 2.8429, 3.8450,
         1.9158, 3.0908, 3.5546, 1.9811, 

In [33]:

# Define the output ONNX file path
onnx_file_path = "mysai.onnx"

# Export the model
torch.onnx.export(
    th_policy_network,                  # Model to export
    input,            # Dummy input
    onnx_file_path,         # Output file path
    export_params=True,     # Export model parameters (weights)
    opset_version=11,       # ONNX opset version (e.g., 11 is widely supported)
    do_constant_folding=True,  # Optimize the model by folding constants
    input_names=["state"],  # Input tensor name
    output_names=["actions"],  # Output tensor name
)

print(f"Model exported to {onnx_file_path}")

Model exported to mysai.onnx
